In [25]:
import json
import numpy as np
from scipy import sparse
from os import listdir
from itertools import product
import matplotlib.pyplot as plt
from collections import namedtuple, defaultdict


In [42]:
data_dir = '../../DATA/platinum'
ped_file = '../../DATA/platinum/platinum.ped.quads.ped'
chroms = [str(x) for x in range(1, 23)]
haplotype_file = '../data/GenotypeFiles/haplotype_transmission_hg19.txt'

#data_dir = '../../DATA/platinum_sim'
#ped_file = '../../DATA/platinum_sim/sim_platinum.ped'
#chroms = ['10']
#haplotype_file = '../simulation/simulated_genomes/simulated_haplotype_transmission_hg19.txt'

gens = ['0/0', '0/1', '1/1']
obss = ['0/0', '0/1', '1/1', './.']



In [43]:
with open('%s/genotypes/info.json' % data_dir, 'r') as f:
	assembly = json.load(f)['assembly']
    
sample_file = '%s/genotypes/samples.json' % data_dir
# pull samples
with open(sample_file, 'r') as f:
	individuals = json.load(f)
sample_id_to_index = dict([(sample_id, i) for i, sample_id in enumerate(individuals)])


In [44]:
children = set()
ind_to_fams = defaultdict(list)

with open(ped_file, 'r') as f:	
    for line in f:
        pieces = line.strip().split('\t')
        if len(pieces) < 4:
            print('ped parsing error', line)
        else:
            fam_id = pieces[0]
            child_id, f_id, m_id = [x.replace('.', '_') for x in pieces[1:4]]
            if f_id != '0' and m_id != '0':
                children.add('%s.%s' % (fam_id, child_id))
            ind_to_fams[child_id].append(fam_id)
            ind_to_fams[m_id].append(fam_id)
            ind_to_fams[f_id].append(fam_id)
def is_child(ind_id):
    return ind_id in children

In [45]:
def pull_gen_data_for_individuals(data_dir, chrom, start_pos=None, end_pos=None, use_pass=False):

	# pull coordinates
	# use only SNPs, no indels
	# use only variants that PASS GATK
	# pull data only for individuals
	m = len(individuals)
	has_seq = np.array(np.where([x in sample_id_to_index for x in individuals])[0].tolist())
	ind_indices = [sample_id_to_index[x] for x in individuals if x in sample_id_to_index]

	if len(ind_indices)==0:
		return np.zeros((m, 0)), np.zeros((0, 2)), np.zeros((0,)) 


	gen_files = sorted([f for f in listdir('%s/genotypes' % data_dir) if ('chr.%s.' % chrom) in f and 'gen.npz' in f], key=lambda x: int(x.split('.')[2]))
	coord_files = sorted([f for f in listdir('%s/genotypes' % data_dir) if ('chr.%s.' % chrom) in f and 'gen.coordinates.npy' in f], key=lambda x: int(x.split('.')[2]))
	#af_files = sorted([f for f in listdir(data_dir) if ('chr.%s.' % chrom) in f and 'gen.af.npy' in f], key=lambda x: int(x.split('.')[2]))

	#print(len(gen_files), len(coord_files))
	assert len(gen_files) == len(coord_files)
	#assert len(gen_files) == len(af_files)

	# pull chrom length
	assert assembly == '37' or assembly == '38'
	with open('../data/chrom_lengths%s.json' % assembly, 'r') as f:     
		chrom_length = json.load(f)[chrom]

	gens, snp_positions, collapseds = [], [], []
	total_pos = 0
	for gen_file, coord_file in zip(gen_files, coord_files):
		coords = np.load('%s/genotypes/%s' % (data_dir, coord_file))

		if coords.shape[0]>0:
			poss = coords[:, 1]
			is_snp = coords[:, 2]==1
			is_pass = coords[:, 3]==1

			if not use_pass:
				is_pass = np.ones(is_pass.shape, dtype=bool)

			# remove multiallelic sites
			multi_indices = np.where(coords[1:, 1]==coords[:-1, 1])[0]
			is_pass[multi_indices] = False
			is_pass[multi_indices+1] = False

			if start_pos is not None and end_pos is not None:
				in_interval = (coords[:, 1]>=start_pos) & (coords[:, 1]<=end_pos)
			else:
				in_interval = np.ones((is_snp.shape[0],), dtype=bool)

			if np.sum(is_snp & is_pass & in_interval)>0:
				gen = sparse.load_npz('%s/genotypes/%s' % (data_dir, gen_file))[ind_indices, :]
				total_pos += np.sum(is_snp & is_pass)
				family_has_variant = ((gen>0).sum(axis=0)>0).A.flatten()

				has_data = np.where(is_snp & is_pass & in_interval & family_has_variant)[0]

				if len(has_data)>0:
				
					# count the number of observed sites in between snps with data
					c = np.cumsum(is_snp & is_pass & in_interval & ~family_has_variant)
					collapsed = np.zeros((len(has_data),), dtype=int)
					collapsed_front = c[has_data[0]]
					collapsed[:-1] = c[has_data][1:]-c[has_data][:-1]
					collapsed[-1] = c[-1]-c[has_data[-1]]
					#print(c[-1]+len(has_data), np.sum(is_snp & is_pass), np.sum(collapsed)+len(has_data)+collapsed_front)

					if len(collapseds) == 0:
						collapseds.append([collapsed_front])
					else:
						collapseds[-1][-1] += collapsed_front

					gens.append(gen[:, has_data].A)
					snp_positions.append(poss[has_data])
					#afs.append(np.digitize(-np.log10(np.clip(af[has_data], 10**-(af_boundaries[0]+1), None)), af_boundaries))
					collapseds.append(collapsed)


	if len(gens)== 0:
		return np.zeros((m, 0)), np.zeros((0, 2)), np.zeros((0,))

	gens = np.hstack(gens)
	snp_positions = np.hstack(snp_positions)
	collapseds = np.hstack(collapseds)

	assert np.all(snp_positions <= chrom_length)
	assert np.all(snp_positions[1:]>=snp_positions[:-1])
	assert np.all(collapseds >= 0)

	n = 2*len(snp_positions)+1
	family_genotypes = np.zeros((len(has_seq), n), dtype=np.int8)
	family_genotypes[:, np.arange(1, n-1, 2)] = gens
		
	observed = np.zeros((n,), dtype=int)
	observed[np.arange(1, n-1, 2)] = 1
	observed[np.arange(0, n, 2)] = collapseds
		
	family_snp_positions = np.zeros((n, 2), dtype=int)
	family_snp_positions[np.arange(1, n-1, 2), 0] = snp_positions
	family_snp_positions[np.arange(1, n-1, 2), 1] = snp_positions+1

	family_snp_positions[np.arange(0, n-2, 2), 1] = snp_positions
	family_snp_positions[np.arange(2, n, 2), 0] = snp_positions+1
	family_snp_positions[0, 0] = 1
	family_snp_positions[-1, 1] = chrom_length

	assert np.all(family_snp_positions[:, 1] >= family_snp_positions[:, 0])

	# remove unnecessary intervals
	haslength = np.where(family_snp_positions[:, 0]!=family_snp_positions[:, 1])[0]
	family_genotypes = family_genotypes[:, haslength]
	family_snp_positions = family_snp_positions[haslength, :]
	observed = observed[haslength]

	# aggregate identical genotypes
	rep_indices = np.where(np.any(family_genotypes[:, 1:]!=family_genotypes[:, :-1], axis=0))[0]
	n = rep_indices.shape[0]+1
	#print('n', n)

	new_family_genotypes = np.zeros((m, n), dtype=np.int8)
	mult_factor = np.zeros((n,), dtype=int)

	new_family_genotypes[np.ix_(has_seq, np.arange(n-1))] = family_genotypes[:, rep_indices]
	new_family_genotypes[has_seq, -1] = family_genotypes[:, -1]

	new_family_snp_positions = np.zeros((n, 2), dtype=int)
	new_family_snp_positions[0, 0] = family_snp_positions[0, 0]
	new_family_snp_positions[:-1, 1] = family_snp_positions[rep_indices, 1]
	new_family_snp_positions[1:, 0] = family_snp_positions[rep_indices+1, 0]
	new_family_snp_positions[-1, 1] = family_snp_positions[-1, 1]

	#print(new_family_snp_positions)

	c = np.cumsum(observed)
	if len(rep_indices)==0:
		mult_factor[0] = np.sum(observed)
	else:
		mult_factor[0] = c[rep_indices[0]]
		mult_factor[1:-1] = c[rep_indices[1:]] - c[rep_indices[:-1]]
		mult_factor[-1] = c[-1] - c[rep_indices[-1]]
	#print(c[-1], np.sum(mult_factor))
	#assert np.all(mult_factor>=0)

	print('genotypes pulled', new_family_genotypes.shape)
	#print(mult_factor)

	return new_family_genotypes, new_family_snp_positions, mult_factor, individuals

In [46]:
def process_bedfile(bed_file, chrom, offset=0):
    regions = []
    coverage = 0
    with open(bed_file, 'r') as f:
        for line in f:
            if '\t' in line.strip():
                pieces = line.strip().split('\t')[offset:]
            else:
                pieces = line.strip().split(':')
                pieces = [pieces[0]] + pieces[1].strip().split('-')

            if pieces[0] == chrom or pieces[0] == 'chr%s' % chrom:
                regions.append(int(pieces[1]))
                regions.append(int(pieces[2])+1)
                coverage += (int(pieces[2])+1 - int(pieces[1]))
    return np.array(regions), coverage


In [48]:
lcr_num = np.zeros((len(individuals), 3, 4))
lcr_denom = np.zeros((len(individuals), 3, 4))
hcr_num = np.zeros((len(individuals), 3, 4))
hcr_denom = np.zeros((len(individuals), 3, 4))
    
for chrom in chroms:
    print(chrom)
    gen, pos, mult, individuals = pull_gen_data_for_individuals(data_dir, chrom)
    gen[gen==-1] = 3
    print('missing', np.sum(gen==3))
    
    # which variants are in LCR?
    lcr_regions, _ = process_bedfile('../data/btu356-suppl_data/btu356_LCR-hs37d5.bed/btu356_LCR-hs37d5.bed', chrom)
    insert_loc = np.searchsorted(lcr_regions, pos[:, 0])
    is_lcr = np.remainder(insert_loc, 2)==1
    
    # pull platinum inheritance pattern
    inheritance_vector = np.zeros((4, len(individuals), gen.shape[1]), dtype=int)
    inh_is_known = np.zeros((gen.shape[1],), dtype=bool)

    with open(haplotype_file, 'r') as f:
        next(f) # skip header
        for line in f:
            pieces = line.strip().split()
            ch = pieces[0][3:]
            if ch == chrom:
                start_pos, end_pos = int(pieces[1]), int(pieces[2])
                inh_is_known[(start_pos <= pos[:, 0]) & (pos[:, 1] <= end_pos)] = True
                for i, inh in enumerate(pieces[3:]):
                    inheritance_vector[0, i, (start_pos <= pos[:, 0]) & (pos[:, 1] <= end_pos)] = 'A' in inh
                    inheritance_vector[1, i, (start_pos <= pos[:, 0]) & (pos[:, 1] <= end_pos)] = 'B' in inh
                    inheritance_vector[2, i, (start_pos <= pos[:, 0]) & (pos[:, 1] <= end_pos)] = 'C' in inh
                    inheritance_vector[3, i, (start_pos <= pos[:, 0]) & (pos[:, 1] <= end_pos)] = 'D' in inh

    # what are the parental variants?
    anc_variants = np.array([x for x in product([0, 1], repeat=4)])
    possible_famgens = np.tensordot(anc_variants, inheritance_vector, axes=(1, 0))
    num_mismatches = np.sum((possible_famgens != np.tile(gen, (16, 1, 1))) & (np.tile(gen, (16, 1, 1)) != 3), axis=1)
    best_anc = np.argmin(num_mismatches, axis=0)
    best_fit = num_mismatches[best_anc, np.arange(gen.shape[1])]
    num_fits = np.sum(num_mismatches == np.tile(best_fit, (16, 1)), axis=0)
    is_good_fit = inh_is_known
    true_gen = possible_famgens[best_anc, :, np.arange(gen.shape[1])].T
    print('no inh', np.sum(~inh_is_known))
    #print('bad fit', np.sum(best_fit>2))
    print(np.sum(is_good_fit & (best_fit != 0)))
    
    # calculate error rates
    
    for j in range(len(individuals)):
        for g in range(len(gens)):
            for obs in range(len(obss)):
                lcr_num[j, g, obs] += np.sum(1/num_fits[is_good_fit & is_lcr & (true_gen[j, :]==g) & (gen[j, :]==obs)])
                lcr_denom[j, g, obs] += np.sum(1/num_fits[is_good_fit & is_lcr & (true_gen[j, :]==g) & (gen[j, :]!=3)])
                hcr_num[j, g, obs] += np.sum(1/num_fits[is_good_fit & ~is_lcr & (true_gen[j, :]==g) & (gen[j, :]==obs)])
                hcr_denom[j, g, obs] += np.sum(1/num_fits[is_good_fit & ~is_lcr & (true_gen[j, :]==g) & (gen[j, :]!=3)])



1
genotypes pulled (13, 761130)
missing 1283
no inh 141
20722
2
genotypes pulled (13, 790262)
missing 1587
no inh 115
16085
3
genotypes pulled (13, 664041)
missing 2677
no inh 96
9329
4
genotypes pulled (13, 715588)
missing 2117
no inh 80
14777
5
genotypes pulled (13, 595848)
missing 1430
no inh 79
5462
6
genotypes pulled (13, 633392)
missing 2353
no inh 60
10092
7
genotypes pulled (13, 559632)
missing 849
no inh 72
13094
8
genotypes pulled (13, 508264)
missing 2627
no inh 79
3895
9
genotypes pulled (13, 436867)
missing 340
no inh 70
16129
10
genotypes pulled (13, 485256)
missing 622
no inh 70
8241
11
genotypes pulled (13, 488173)
missing 2600
no inh 54
5261
12
genotypes pulled (13, 456194)
missing 960
no inh 63
4705
13
genotypes pulled (13, 365903)
missing 399
no inh 53
3978
14
genotypes pulled (13, 301898)
missing 789
no inh 45
4750
15
genotypes pulled (13, 285210)
missing 395
no inh 58
4661
16
genotypes pulled (13, 306744)
missing 1566
no inh 58
14082
17
genotypes pulled (13, 266345

In [49]:
print(lcr_num.shape)

(13, 3, 4)


In [52]:
# write error rate estimates
with open('%s/sequencing_error_rates/LCR_errors.json' % data_dir, 'w+') as f:
    error_rates = defaultdict(dict)
    for index, ind in enumerate(individuals):
        for fam in ind_to_fams[ind]:
            for i, gen in enumerate(gens):
                total_prob = 0
                for j, obs in enumerate(obss):
                    if gen != obs:
                    
                        if obs == './.':
                            p = np.sum(lcr_num[index, :, j])/np.sum(lcr_denom[index, :, j])
                        elif lcr_num[index, i, j]==0:
                            p = np.sum(lcr_num[:, i, j])/np.sum(lcr_denom[:, i, j])                  
                        else:
                            p = lcr_num[index, i, j]/lcr_denom[index, i, j]
                        error_rates['%s.%s' % (fam, ind)]['-log10(P[obs=%s|true_gen=%s])' % (obs, gen)] = -np.log10(p)
                        total_prob += p
                    
    
                # make sure total prob sums to 1
                assert total_prob<1
                error_rates['%s.%s' % (fam, ind)]['-log10(P[obs=%s|true_gen=%s])' % (gen, gen)] = -np.log10(1-total_prob)

    json.dump(error_rates, f, indent=4)
    
with open('%s/sequencing_error_rates/HCR_errors.json' % data_dir, 'w+') as f:
    error_rates = defaultdict(dict)
    for index, ind in enumerate(individuals):
        for fam in ind_to_fams[ind]:
            for i, gen in enumerate(gens):
                total_prob = 0
                for j, obs in enumerate(obss):
                    if gen != obs:
                        if obs == './.':
                            p = np.sum(hcr_num[index, :, j])/np.sum(hcr_denom[index, :, j])
                        elif hcr_num[index, i, j]==0:
                            p = np.sum(hcr_num[:, i, j])/np.sum(hcr_denom[:, i, j])               
                        else:
                            p = hcr_num[index, i, j]/hcr_denom[index, i, j]
                        error_rates['%s.%s' % (fam, ind)]['-log10(P[obs=%s|true_gen=%s])' % (obs, gen)] = -np.log10(p)
                        total_prob += p
                    
                # make sure total prob sums to 1
                assert total_prob<1
                error_rates['%s.%s' % (fam, ind)]['-log10(P[obs=%s|true_gen=%s])' % (gen, gen)] = -np.log10(1-total_prob)

            
    json.dump(error_rates, f, indent=4)
  